### Linear Regression & Scikit-Learn Intro

This lab is designed to give everyone their first introduction to the Scikit-Learn API, and Linear Regression, one of the most commonly used techniques in predictive modeling.

During this lab you'll see if you can build a model, understand its working parts, and make improvements to your results!  

The great thing about `Scikit Learn` is that its API is almost identical from one algorithm to another, so once you get the hang of how to use it, using different methods is fairly seamless.

**Step 1:** Load in the `housing.csv` file

In [2]:
import pandas as pd
df = pd.read_csv('../../data/housing.csv')

**Step 2:** What columns to include?  This is a bit of a topic unto itself, but for now let's keep it fairly simple, and choose the 5 columns that have the highest correlation (in magnitude) to the `PRICE` column. 

**hint:** `df.corr()`, and/or `sns.heatmap()` are helpful here.

In [8]:
import seaborn as sns
top5 = df.corr()['PRICE'].abs().sort_values(ascending=False)[1:6]

**Step 3:** Declare your `X` & `y` variables

In [11]:
# from sklearn. import train_test_split
X = df[top5.index]
y = df['PRICE']

**Step 4:** Import `LinearRegression` and initialize it.

In [12]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

**Step 5:** Call the `fit()` method on `X` & `y`

In [13]:
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Step 6:** Look at the values for `lreg.coef_` & `lreg.intercept_`

In [14]:
print(lr.coef_)
print(lr.intercept_)

[-5.59004744e-01  4.62516926e+00 -8.76154330e-01  5.69752124e-02
 -3.53693090e-03]
17.51771284955599


**Step 7:** Turn your model output into a more readable format.  

Try using the following:

  `coeff_dict = {'Column': X.columns, 'Weight': lreg.coef_}`
  
  `coeff = pd.DataFrame(coeff_dict)`

In [16]:
coeff_dict = {'Column': X.columns, 'Weight': lr.coef_}
coeff = pd.DataFrame(coeff_dict)
coeff

,Column,Weight
0,LSTAT,-0.559005
1,RM,4.625169
2,PTRATIO,-0.876154
3,INDUS,0.056975
4,TAX,-0.003537


**Step 8:** Using the table you created in the previous step, what column had the largest coefficient (in magnitude)?

Does this mean it has the most predictive value of any column in your dataset? (We'll discuss this in class).

In [24]:
coeff.iloc[(coeff['Weight'].abs().idxmax())]

Column         RM
Weight    4.62517
Name: 1, dtype: object

**Step 9:** What is your model's score? (We haven't really talked about what this means yet.  Don't worry, we'll get to that).

In [27]:
lr.score(X, y)

0.6804097741290724

**Step 10:** What is the equation of your model?

Ie, `y = coefval1 * RM + coefval2 * TAX + coefval3 * SomeColumnName + intercept`

In [34]:
# print('y = {:.2f} * {:s} + {:.2f} * {:s} + {:.2f} * {:s} + {:.2f} * {:s} + {:.2f} * {:s}'.format(coeff[0].Column))

**Step 11:** Make a column in your dataframe called `PREDICTION`, and make it the predicted values of each row in `X`.

In [32]:
df['PREDICTION'] = lr.predict(X)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTION
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,30.823877
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,26.057952
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,32.448095
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,31.196095
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,30.549224


**Step 12:** Looking at your first observation, can you make sense of the predicted value?  Ie, how it was generated from your coefficients & intercept?  If you're the type to do this, see if you can recreate the prediction using just your samples, coefficients and intercept.

In [38]:
import numpy as np
np.dot(df.loc[0, top5.index], lr.coef_) + lr.intercept_

30.8238770801465

#### Bonus

If you've made it this far, good job.  You can congratulate yourself for having duly completed the meat of today's lesson.  However, there are more details to discuss with these things, and these questions will give you a head's up on future points of attention.

**Bonus 1:** Data Scaling

The literal meaning of a coefficient is it's the expected change in `y` you would get by increasing its associated column's value by exactly 1.  

What's problematic about this definition is that most of the time data exists on different scales, making the meaning of your coefficients less clear.  

For example, the `TAX` column has a range of 200-800, and the `NOX` column has a range of 0.3-0.8.  Increasing the values in these columns by 1 will necessarily mean very different things since their scales are so different.  
To get around this, you typically scale your data before feeding it to a Linear Regression model (or anything else that uses coefficients). 

You accomplish this by taking every column and subtracting its mean from every value, and then dividing each value in a column by its standard deviation.  

You can do it like this:

`X = X - X.mean()`

`X = X / X.std()`

**a). Standardize Your Data, And Use the `Describe()` method to verify your results**

In [40]:
X = X - X.mean()
X = X / X.std()
X.describe().round(5)

,LSTAT,RM,PTRATIO,INDUS,TAX
count,506.00000,506.00000,506.00000,506.00000,506.00000
mean,0.00000,-0.00000,-0.00000,-0.00000,-0.00000
std,1.00000,1.00000,1.00000,1.00000,1.00000
min,-1.52961,-3.87641,-2.70470,-1.55630,-1.31269
25%,-0.79863,-0.56807,-0.48756,-0.86683,-0.76682
50%,-0.18107,-0.10836,0.27459,-0.21089,-0.46421
75%,0.60242,0.48229,0.80578,1.01499,1.52941
max,3.54526,3.55153,1.63721,2.42017,1.79642


**b). Refit your model on `X` and `y`, and look at your new coefficients.  Are they they the same or different?**

In [42]:
lr2 = LinearRegression()
lr2.fit(X, y)
print(lr.coef_, lr2.coef_, sep='\n')

[-5.59004744e-01  4.62516926e+00 -8.76154330e-01  5.69752124e-02
 -3.53693090e-03]
[-3.99188726  3.24972322 -1.8968264   0.39087007 -0.59610413]


**c).  Look at your score.  Is it different?**

In [43]:
print(lr2.score(X, y))

0.6804097741290724


**d). What about your predictions?**

In [45]:
df['PREDICTION2'] = lr2.predict(X)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTION,PREDICTION2
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,30.823877,30.823877
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,26.057952,26.057952
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,32.448095,32.448095
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,31.196095,31.196095
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,30.549224,30.549224


**Bonus II: Basis Expansion**

The most common problem with Linear Regression models is that data is not always so linear.  Sometimes there are increasing returns to scale, diminishing returns, outliers, or erratic jump points that make your results untrustworthy.  

One common way to get around this is something called **basis expansion**, which is simply finding ways to capture irregular patterns in your data via different types of numeric transformations.  

In fact, lots of cutting edge techniques can be accurately framed as basis expansions on linear models.  

In this section, we'll see if we can find some low-hanging fruit in the columns that we currently have to perhaps better handle not-so-linear patterns. 

Here are some things you can try:

 - If the relationship between two columns looks like it's exponentially increasing or parabolic (bowl shaped), you can try raising the number to a higher power.  Ie, `df[col]**2 or df[col]**3`, etc.
 - If it looks like you are hitting diminishing returns you can try taking the square-root of a column and adding that to your mix of predictors
 - If a column looks like it's plagued by irregular values then you can use a log transformation `(np.log(df[col]))` to make it more numerically stable
 - If you think two columns (or more) had a multiplicative effect then you can try creating a new column that's the product of those columns multiplied together and use that in your regression.  These are called *interaction effects*.  Ie, do things like `CRIM` and `LSTAT` (socioeconomic status) reinforce one another?  If so, multiplying them together can capture their joint impact.
 
You could potentially do any of these to any columns in `X` or `y`.

With that said, try using either the `pairplot` or `regplot`, or some other data visualization tool to see if there's perhaps a higher order relationship to be captured.  If you see something that looks intriguing, add a new column to `X` that captures this relationship, re-run your regression, and see if your model score improved.

In [55]:
# sns.pairplot(df)

**Bonus III: Regularized Regression**

Many times you'll have a dataset that has many columns that are highly correlated, or just of dubious quality and you're better off excluding them from your model.  

In these circumstances, it can be difficult to get reliable results that replicate well to out-of-sample data, or reason about what to even include for your model.

Two common variants of common Linear Regression which help with these issues are **Ridge Regression** and **Lasso Regression**.  

They serve the same general purpose of reducing the impact of multicollinearity (highly correlated columns) and/or removal of low quality columns.  However, how they go about doing so is slightly different.

For now, let's just see how they work.

**Run the following cell to import Ridge and Lasso Regression**

In [49]:
# import the models here
from sklearn.linear_model import Ridge, Lasso

**Initialize instances of both, and fit them on your data.**

In [50]:
ridge = Ridge()
ridge.fit(X, y)
lasso = Lasso()
lasso.fit(X, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

**Look at the values of your coefficients.  How are they different from regular Linear Regression?**

In [51]:
print(lr.coef_)
print(lr2.coef_)
print(ridge.coef_)
print(lasso.coef_)

[-5.59004744e-01  4.62516926e+00 -8.76154330e-01  5.69752124e-02
 -3.53693090e-03]
[-3.99188726  3.24972322 -1.8968264   0.39087007 -0.59610413]
[-3.98073675  3.24729699 -1.89457005  0.38146023 -0.59595601]
[-3.62698295  2.68460445 -1.35715355 -0.         -0.        ]


If you look at the parameters of each one, you'll notice they have something called `alpha`.  This is a value that can go as low as 0 and has no upper limit.  

**Try changing the values of alpha for both Ridge and Lasso by multiples of 10, and notice what impact it has on your coefficients. (You can try values that are less than 1.  This is especially useful for Lasso, which is more sensitive to alpha than Ridge).**

In [52]:
ridge = Ridge(alpha=10)
ridge.fit(X, y)
print(ridge.coef_)
lasso = Lasso(alpha=10)
lasso.fit(X, y)
print(lasso.coef_)

[-3.88514938  3.22450024 -1.87461405  0.30183334 -0.59579964]
[-0.  0. -0. -0. -0.]


In [53]:
ridge = Ridge(alpha=.10)
ridge.fit(X, y)
print(ridge.coef_)
lasso = Lasso(alpha=.10)
lasso.fit(X, y)
print(lasso.coef_)

[-3.99076717  3.24948166 -1.89660041  0.3899237  -0.59608801]
[-3.87227901  3.1667798  -1.84494445  0.         -0.32738203]


In [54]:
ridge = Ridge(alpha=20)
ridge.fit(X, y)
print(ridge.coef_)
lasso = Lasso(alpha=.03)
lasso.fit(X, y)
print(lasso.coef_)

[-3.78794661  3.19752618 -1.85317036  0.22282325 -0.59761998]
[-3.95075313  3.22340456 -1.88107096  0.25331602 -0.5042252 ]
